In [1]:
# ==========================================
# File: project_cdft/notebooks/01_feature_engineering.ipynb
# ==========================================
# [CDFT 专属] 特征工程 (物理纯净版)
# 设计哲学:
# 1. 低维强先验: 不做相关性剔除，保留所有物理描述符
# 2. 稳定性优先: 仅做基础清洗 (去空、去单一值)
# 3. 极简主义: 无复杂的特征变换，保持物理意义可解释

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 1. 环境定位
if 'notebooks' in os.getcwd(): os.chdir('..')
print(f"🚀 [01] 启动 CDFT 特征工程 (Physics Mode) | 目录: {os.getcwd()}")

# ========================================================
# [Step 1] 读取数据
# ========================================================
raw_dir = 'data/raw'
if not os.path.exists(raw_dir): raise FileNotFoundError(f"❌ 找不到 {raw_dir}")
csv_files = [f for f in os.listdir(raw_dir) if f.lower().endswith('.csv')]
if not csv_files: raise FileNotFoundError(f"❌ {raw_dir} 为空")

# 读取文件并去除列名空格
df = pd.read_csv(os.path.join(raw_dir, csv_files[0]))
df.columns = df.columns.str.strip()

print(f"   读取原始数据: {df.shape}")

# ========================================================
# [Step 2] 智能识别 Target 与 SMILES
# ========================================================
# 识别 Target (优先找 log 相关的，或 Activity/Target)
target_col = None
candidates = ['Target_Log1o2', 'log(1o2)', 'Target', 'Activity', 'pIC50', 'Y']
for c in candidates:
    if c in df.columns: target_col = c; break

# 兜底：如果没有找到标准名，且最后一列是数值，就默认为 Target
if not target_col:
    target_col = df.columns[-1]
    print(f"⚠️ 未匹配到标准 Target 名，默认使用最后一列: {target_col}")

# 识别 SMILES (用于 Meta 信息，不参与训练)
smiles_col = 'SMILES' if 'SMILES' in df.columns else df.columns[0]

print(f"✅ 锁定 Target: [{target_col}]")
print(f"✅ 锁定 Meta/SMILES: [{smiles_col}]")

# ========================================================
# [Step 3] 极简特征清洗 (Minimal Cleaning)
# ========================================================
# 分离数据
y = df[target_col]
meta = df[smiles_col]

# 提取特征 (排除非数值列、Target、SMILES、以及 ID/Name 等无关列)
exclude_cols = [target_col, smiles_col, 'ID', 'No.', 'Name', 'Molecule', 'Compound']
X = df.drop(columns=exclude_cols, errors='ignore').select_dtypes(include=[np.number])

# 1. 基础修补 (CDFT 计算失败可能产生 NaN/Inf)
# 策略: 填充中位数 (保证鲁棒性)，而不是直接删除行 (数据宝贵)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
if X.isnull().sum().sum() > 0:
    print(f"   ⚠️ 检测到缺失值，正在使用中位数填充...")
    X.fillna(X.median(), inplace=True)
    X.fillna(0, inplace=True) # 防止全列 NaN 的情况

# 2. 去除单一值 (Zero Variance)
# 如果某列所有值都一样，对模型无用，必须删
start_cols = X.shape[1]
X = X.loc[:, (X != X.iloc[0]).any()]
dropped_cols = start_cols - X.shape[1]

# ★ 关键修改: 不做相关性剔除 ★
# CDFT 特征每一个都有物理含义，即使相关也不删除，留给模型去处理。

print(f"✅ 特征清洗完成: {X.shape}")
print(f"   - 删除了 {dropped_cols} 个单一值特征")
print(f"   - 保留了所有物理描述符 (不做去相关)")

# ========================================================
# [Step 4] 动态切分 (Seed from 05)
# ========================================================
seed_file = "current_seed_config.txt"
if os.path.exists(seed_file):
    SEED = int(open(seed_file).read().strip())
    print(f"📨 [指令] 使用 Seed = {SEED}")
else:
    SEED = 888
    print(f"🛑 [默认] 使用 Seed = {SEED}")

# 组合临时数据用于切分
data = X.copy()
data['Target_Log1o2'] = y
data['SMILES_Meta'] = meta

# 切分 70/15/15
train_val, test = train_test_split(data, test_size=0.15, random_state=SEED)
train, val = train_test_split(train_val, test_size=15/85, random_state=SEED)

# ========================================================
# [Step 5] 保存
# ========================================================
os.makedirs('data/processed', exist_ok=True)
train.to_csv('data/processed/train.csv', index=False)
val.to_csv('data/processed/val.csv', index=False)
test.to_csv('data/processed/test.csv', index=False)

print(f"💾 数据已保存至 data/processed/ (Seed {SEED})")
print(f"   Train: {train.shape[0]} | Val: {val.shape[0]} | Test: {test.shape[0]}")

🚀 [01] 启动 CDFT 特征工程 (Physics Mode) | 目录: D:\HO\my_smiles_project\y_g.baby\project_cdft
   读取原始数据: (151, 33)
✅ 锁定 Target: [log(1o2)]
✅ 锁定 Meta/SMILES: [SMILES]
✅ 特征清洗完成: (151, 23)
   - 删除了 1 个单一值特征
   - 保留了所有物理描述符 (不做去相关)
📨 [指令] 使用 Seed = 288
💾 数据已保存至 data/processed/ (Seed 288)
   Train: 105 | Val: 23 | Test: 23
